In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
train = pd.read_csv("train_data.csv")
train.drop(["Id","date"],axis=1,inplace=True)
train = train.values
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms.fit(train)
train = mms.transform(train)
labels = pd.read_csv("train_target.csv")
labels = labels["target"].to_numpy()
labels,train

(array([1, 0, 1, ..., 1, 0, 0], dtype=int64),
 array([[0.28775342, 0.28420792, 0.28852409, ..., 0.27166192, 0.27187271,
         0.27825302],
        [0.28247401, 0.28006026, 0.29070949, ..., 0.27150946, 0.27135311,
         0.27846074],
        [0.28357518, 0.2834341 , 0.29248831, ..., 0.27154653, 0.27150045,
         0.27838975],
        ...,
        [0.80950911, 0.80827882, 0.81252287, ..., 0.8060414 , 0.8071335 ,
         0.80491935],
        [0.80901513, 0.80812406, 0.81252287, ..., 0.80641053, 0.80740769,
         0.80537639],
        [0.80959144, 0.80823755, 0.81279732, ..., 0.8069243 , 0.80726751,
         0.80652161]]))

In [42]:
windows = []
horizon = []
i = 0
while(i+7<len(train)):
    lab = []
    lab.append(train[i])
    lab.append(train[i+1])
    lab.append(train[i+2])
    lab.append(train[i+3])
    lab.append(train[i+4])
    lab.append(train[i+5])
    lab.append(train[i+6])
    windows.append(lab)
    horizon.append(labels[i+7])
    i+=1
print(np.array(windows).shape)
print(np.array(horizon).shape)  
windows = np.array(windows)
horizon = np.array(horizon)

(25493, 7, 16)
(25493,)


In [43]:
from tensorflow.keras.layers import Input, Conv1D, Dense, Lambda, Flatten, Concatenate
from tensorflow.keras import Model
from tensorflow.keras import metrics
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras import utils
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

        
inputs = Input(shape=(7,16))

# extract our features using a Convolutional layers, hence "CNN"
feature_extraction = Conv1D(64, 2, activation='relu')(inputs)
# split layer into three regions based on time, hence "Temporal"
long_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[0])(feature_extraction)
mid_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[1])(feature_extraction)
short_term = Lambda( lambda x: tf.split(x, num_or_size_splits=3, axis=1)[2])(feature_extraction)

long_term_conv = Conv1D(1, 1, activation='relu')(long_term)
mid_term_conv = Conv1D(1, 1, activation='relu')(mid_term)
short_term_conv = Conv1D(1, 1, activation='relu')(short_term)

# combine three layers back into one
combined = Concatenate(axis=1)([long_term_conv, mid_term_conv, short_term_conv])

# flattening is required since our input is a 2D matrix
flattened = Flatten()(combined)

# 1 output neuron for each class (Up, Stationary, Down --- see Direction class)
x = Dense(20, activation='softmax')(flattened)
outputs = Dense(1,activation="sigmoid")(x)
# specify input and output layers of our model
model = Model(inputs=inputs, outputs=outputs)

# compile our model
model.compile(optimizer='adam',
              loss="binary_crossentropy",metrics="accuracy")
model.fit(windows, horizon, epochs=50)

Epoch 1/50
797/797 [==============================] - 36s 6ms/step - loss: 0.6929 - accuracy: 0.5086
Epoch 2/50
797/797 [==============================] - 4s 5ms/step - loss: 0.6928 - accuracy: 0.5080
Epoch 3/50
797/797 [==============================] - 4s 6ms/step - loss: 0.6926 - accuracy: 0.5129
Epoch 4/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6924 - accuracy: 0.5121
Epoch 5/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6925 - accuracy: 0.5136
Epoch 6/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6925 - accuracy: 0.5128
Epoch 7/50
797/797 [==============================] - 4s 6ms/step - loss: 0.6924 - accuracy: 0.5173
Epoch 8/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6924 - accuracy: 0.5156
Epoch 9/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6923 - accuracy: 0.5167
Epoch 10/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6924 - accuracy: 0.518

In [44]:
model.fit(windows, horizon, epochs=50)

Epoch 1/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6917 - accuracy: 0.5203
Epoch 2/50
797/797 [==============================] - 4s 5ms/step - loss: 0.6917 - accuracy: 0.5198
Epoch 3/50
797/797 [==============================] - 4s 5ms/step - loss: 0.6918 - accuracy: 0.5192
Epoch 4/50
797/797 [==============================] - 4s 6ms/step - loss: 0.6918 - accuracy: 0.5192
Epoch 5/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6917 - accuracy: 0.5180
Epoch 6/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6917 - accuracy: 0.5203
Epoch 7/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6918 - accuracy: 0.5208: 0s - loss: 0.6918 - ac
Epoch 8/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6917 - accuracy: 0.5221
Epoch 9/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6917 - accuracy: 0.5199
Epoch 10/50
797/797 [==============================] - 5s 6ms/step - loss: 0

In [57]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss="binary_crossentropy",metrics="accuracy")

In [53]:
model.fit(windows, horizon, epochs=50)

Epoch 1/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5211
Epoch 2/50
797/797 [==============================] - 4s 5ms/step - loss: 0.6912 - accuracy: 0.5218
Epoch 3/50
797/797 [==============================] - 4s 6ms/step - loss: 0.6912 - accuracy: 0.5199
Epoch 4/50
797/797 [==============================] - 4s 6ms/step - loss: 0.6912 - accuracy: 0.5226
Epoch 5/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5198
Epoch 6/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6911 - accuracy: 0.5242
Epoch 7/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5212
Epoch 8/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6912 - accuracy: 0.5234
Epoch 9/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6911 - accuracy: 0.5213
Epoch 10/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6911 - accuracy: 0.5205

KeyboardInterrupt: 

In [48]:
model.fit(windows, horizon, epochs=50)

Epoch 1/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5248
Epoch 2/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5250
Epoch 3/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5249
Epoch 4/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5243
Epoch 5/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5246
Epoch 6/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5248
Epoch 7/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5247
Epoch 8/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5249
Epoch 9/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5252
Epoch 10/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5245

In [58]:
model.fit(windows, horizon, epochs=50)

Epoch 1/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5245
Epoch 2/50
797/797 [==============================] - 5s 6ms/step - loss: 0.6906 - accuracy: 0.5252
Epoch 3/50
797/797 [==============================] - 5s 7ms/step - loss: 0.6906 - accuracy: 0.5248
Epoch 4/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6906 - accuracy: 0.5245
Epoch 5/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6906 - accuracy: 0.5249
Epoch 6/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6906 - accuracy: 0.5246
Epoch 7/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6906 - accuracy: 0.5250
Epoch 8/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6906 - accuracy: 0.5247
Epoch 9/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6906 - accuracy: 0.5251
Epoch 10/50
797/797 [==============================] - 8s 10ms/step - loss: 0.6906 - accuracy:

In [51]:
model.save("model1.h5")

In [56]:
model = tf.keras.models.load_model('model1.h5')

In [60]:
test = pd.read_csv("test_data.csv")
test.drop(["Id","date"],axis=1,inplace=True)
test = test.values
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms.fit(test)
test = mms.transform(test)
test.shape

(10317, 16)

In [70]:
windows_test = []
i = 0
while(i+6<len(test)):
    lab = []
    lab.append(test[i])
    lab.append(test[i+1])
    lab.append(test[i+2])
    lab.append(test[i+3])
    lab.append(test[i+4])
    lab.append(test[i+5])
    lab.append(test[i+6])
    windows_test.append(lab)
    i+=1
print(np.array(windows_test).shape)  
windows_test = np.array(windows_test)

(10311, 7, 16)


In [71]:
preds = model.predict(windows_test)

In [72]:
preds = tf.round(preds).numpy()
preds

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [73]:
preds = tf.cast(preds,dtype=tf.int64).numpy()
preds

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [74]:
preds = tf.squeeze(preds).numpy()
preds

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [78]:
l = np.arange(0,len(preds))
pred_df = pd.DataFrame(list(zip(l, preds)),
               columns =['Id', 'target'])
pred_df.to_csv("submission10.csv")
pred_df["target"].value_counts()

1    7632
0    2679
Name: target, dtype: int64